In [1]:
import cv2 #opencv
import mediapipe as mp
import math
import csv
import datetime
import pandas as pd
import os
from mediapipe_estimate import estimate
pd.options.mode.chained_assignment = None 

In [2]:
test_dat=data=pd.read_csv('golfDB_front_orig.csv',index_col='id')
test_dat=test_dat[['youtube_id','events_2','events_7','events_9']]
test_dat['events_9']=test_dat['events_9']-test_dat['events_2']+15
test_dat['events_7']=test_dat['events_7']-test_dat['events_2']+15
test_dat=test_dat.drop(['events_2'],axis=1)
test_dat

,youtube_id,events_7,events_9
id,,,
8,-M5SITXMA2Y,53,69
10,gOBVh7fzyZo,47,64
14,vN3Uc_EhnnY,49,64
19,iW323nsTGtU,57,82
21,xD6KDqPF9cc,45,66
...,...,...,...
1370,g90r9cs1tTw,43,65
1372,eNlBbMI-MhQ,59,81
1374,6K1FD0sOwoI,63,86


In [3]:
folder_path = '23206-final-pose-estimation-for-swing-improvement/cropped_videos'
name_list=list('output_video_'+pd.unique(test_dat.youtube_id))
all=len(name_list)
correct=0
for fl in name_list:
    try:
        data=pd.read_csv('results/variables_'+fl+'.csv')
        
        #consider only time between backswing and finish
        halfway_back_ind=data['right_wrist_x'].idxmin()
        halfway_up_ind=data.right_wrist_x[data.index>halfway_back_ind].idxmax()
        middle_data=data[(data.index>halfway_back_ind)&(data.index<halfway_up_ind)]

        #find moment of ball contact as tha maximum of wrist y 
        contact_frame=middle_data[middle_data['right_wrist_y']==middle_data['right_wrist_y'].max()].index
        

        
        #extract the gt from golfDB
        yt_id='_'.join(fl.split('_')[2::])
        gt=test_dat[test_dat.youtube_id==yt_id].events_7.item()

        #calculte tolerance as per golfDB paper
        tolerance=2*(gt//fps)


        if contact_frame.item()<=tolerance+gt and contact_frame.item()>=gt-tolerance:
            correct+=1
            print('correct esitmate',correct,' estimated [s]:',data[data.index==contact_frame.item()].timestamp.item(),'gt [s]:', data[data.index==test_dat[test_dat.youtube_id==fl.split('_')[2]].events_7.item()].timestamp.item(),', tolerance [frames]:',tolerance, ', frame difference:', abs(contact_frame.item()-gt))

        else:
            print('incorrect estimate',correct,' estimated [s]:',data[data.index==contact_frame.item()].timestamp.item(),', gt [s]:', data[data.index==test_dat[test_dat.youtube_id==fl.split('_')[2]].events_7.item()].timestamp.item(),', tolerance [frames]:',tolerance,', frame difference:', abs(contact_frame.item()-gt))
    except Exception as e:
        print(e,fl)
        all=all-1
        
              
            
accuracy=correct/len(name_list)
print(accuracy)

incorrect estimate 0  estimated [s]: 1.43 , gt [s]: 1.77 , tolerance [frames]: 2 , frame difference: 10
correct esitmate 1  estimated [s]: 1.53 gt [s]: 1.57 , tolerance [frames]: 2 , frame difference: 1
attempt to get argmin of an empty sequence output_video_vN3Uc_EhnnY
correct esitmate 2  estimated [s]: 1.87 gt [s]: 1.9 , tolerance [frames]: 2 , frame difference: 1
correct esitmate 3  estimated [s]: 1.47 gt [s]: 1.53 , tolerance [frames]: 2 , frame difference: 2
can only convert an array of size 1 to a Python scalar output_video_pxO_eGmiDFk
can only convert an array of size 1 to a Python scalar output_video_2Us2Zx_S6hI
correct esitmate 6  estimated [s]: 1.83 gt [s]: 1.83 , tolerance [frames]: 2 , frame difference: 0
can only convert an array of size 1 to a Python scalar output_video_j6h-3Ez0714
incorrect estimate 6  estimated [s]: 1.83 , gt [s]: 1.93 , tolerance [frames]: 2 , frame difference: 3
correct esitmate 7  estimated [s]: 1.77 gt [s]: 1.83 , tolerance [frames]: 2 , frame diffe

KeyboardInterrupt: 

: 

In [ ]:
# data=pd.read_csv('results/variables_'+fl+'.csv')
# halfway_back_ind=data['right_wrist_x'].idxmin()
# data_back=data[data.index>halfway_back_ind]
# data['sum_wrists_hit']=data['right_wrist_y']+data.timestamp
# ev_6=data[data['sum_wrists_hit']==data['sum_wrists_hit'].max()].index

In [ ]:
# data=pd.read_csv('results/variables_'+fl+'.csv')
